In [1]:
import pika
import uuid
import random
import json
from datetime import datetime, timezone

In [2]:
rabbitmq_host = 'localhost'
rabbitmq_port = 5672  # Default RabbitMQ port
rabbitmq_username = 'jackpotAdmin'
rabbitmq_password = 'jackpotPassword'
rabbitmq_queue = 'reward-queue'

In [3]:
credentials = pika.PlainCredentials(username=rabbitmq_username, password=rabbitmq_password)
connection_params = pika.ConnectionParameters(host=rabbitmq_host, port=rabbitmq_port, credentials=credentials)

In [6]:
experiment_id = "2-feats-eg"
arms = ["option1", "option2", "option3", "option4"]

In [136]:
# Send arm pulled and reward obtained messages

In [7]:
def get_current_utc_timestamp():
    current_utc_time = datetime.utcnow().replace(tzinfo=timezone.utc)
    formatted_timestamp = current_utc_time.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
    return formatted_timestamp

In [8]:
from time import sleep
for i in range(10000): 
    sleep(0.2)
    arm_selected = random.choices(arms, weights=(10, 10, 10, 70), k=1)[0]
    sess_id = uuid.uuid4()
    reward = random.choice([1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

    same_timestamp = get_current_utc_timestamp()

    message_body = {
        "reward": {
            "arm_selected": arm_selected,
            "experiment_id": experiment_id,
            "session_id": str(sess_id),
            "timestamp": same_timestamp,
            "context": {
                "age": "90",
                "is_female": "1"
            }
        },
        "operation": "create"
    }

    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()
    channel.basic_publish(exchange='', routing_key=rabbitmq_queue, body=json.dumps(message_body))

    if reward == 1:
        collected_reward_message_body = {
            "reward": {
                "experiment_id": experiment_id,
                "session_id": str(sess_id),
                "reward": reward,
                "arm_selected": arm_selected,
                "timestamp": same_timestamp
            },
            "operation": "update"
        }

        connection = pika.BlockingConnection(connection_params)
        channel = connection.channel()
        channel.basic_publish(exchange='', routing_key=rabbitmq_queue, body=json.dumps(collected_reward_message_body))

KeyboardInterrupt: 

In [37]:
connection.close()